In [1]:
from importlib import reload
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import environment as en
import agent as ag
import superviser as sv

%matplotlib inline

# 1.	Получение данных

In [2]:
type_insrt = 'pamm'#'pif'

# Количество рабочих дней (строк в БД) в периоде (году)
year_days = 260

# Минимальное количество рабочих дней, для которых возможно посчитать доходность за минимальный период
min_hist_days = 260 # 124

# Минимальный период расчёта рентабельности для последующего перевода в годовую рентабельность
min_rent_days = 260 # 124

In [3]:
reload(en)
envir = en.Environment(date_start = pd.to_datetime('2012-06-01'), cash_start = 1000000, type_insrt = type_insrt, file_extension = 'xls')

In [4]:
reload(ag)
agent = ag.Agent(envir)

In [5]:
reload(sv)
suvis = sv.Superviser(envir, agent, type_insrt = type_insrt)

In [6]:
result_df = pd.DataFrame(columns=['date_start', 'top_thres', 'pow_st_dev', 'den_pow', 'port_summ_value'])
metadata_df = pd.DataFrame(columns=['parameter', 'value'])

metadata_df = metadata_df.append({'parameter': 'selection method', 'value': 'top'}, ignore_index = True)
metadata_df = metadata_df.append({'parameter': 'bet_thres^pow', 'value': '1'}, ignore_index = True)

for year in range(2017, 2018): #2009, 2015
    date_start = pd.to_datetime(str(year) + '-06-01')
    date_finish = pd.to_datetime(str(year) + '-06-01') # + 1 
    #for min_thres in np.arange(1.10, 1.20, 0.01):
    for top_thres in np.arange(4, 5, 1):
        for pow_st_dev in np.arange(0.5, 0.6, 0.1):
            for den_pow in [260]: #     124, 248, 496
                print ('\r','date_start: ', str(year) + '-06-01', 'top_thres: ' , top_thres, ' pow_st_dev: ', pow_st_dev, 'den_pow: ', den_pow, end ='') #, min_thres, 'pow_st_dev: '
                #suvis.action()
                envir.start(date_start = date_start, 
                            date_finish = date_finish)
                agent.start(top_thres = top_thres, 
                            pow_st_dev = pow_st_dev, 
                            den_pow = den_pow, 
                            year_days = year_days,
                            min_hist_days = min_hist_days,
                            min_rent_days = min_rent_days) #, min_thres = min_thres
                agent.action()
                #print ('\nИТОГО: ', "{:.0f}".format(agent.port_summ_value))

                result_df = result_df.append({'date_start': date_start, 'top_thres': top_thres, 'pow_st_dev': pow_st_dev, 'den_pow': den_pow, 'port_summ_value': agent.port_summ_value}, ignore_index = True) #, 'min_thres': min_thres
            
# Сохранение истории поиска по сетке
writer = pd.ExcelWriter('..\\5. Evaluation\\' + type_insrt + '\\result_grid_search.xlsx')
metadata_df.to_excel(writer, 'metadata', header = False, index = False)
result_df.to_excel(writer, 'data')
writer.save()

writer = pd.ExcelWriter('..\\5. Evaluation\\' + type_insrt + '\\result_grid_search ' + datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S") + '.xlsx')
metadata_df.to_excel(writer, 'metadata', header = False, index = False)
result_df.to_excel(writer, 'data')
writer.save()

 date_start:  2017-06-01 top_thres:  4  pow_st_dev:  0.5 den_pow:  260

In [7]:
# Сохранение истории поиска по сетке
writer = pd.ExcelWriter('..\\5. Evaluation\\' + type_insrt + '\\result_grid_search.xlsx')
metadata_df.to_excel(writer, 'metadata', header = False, index = False)
result_df.to_excel(writer, 'data')
writer.save()

writer = pd.ExcelWriter('..\\5. Evaluation\\' + type_insrt + '\\result_grid_search ' + datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S") + '.xlsx')
metadata_df.to_excel(writer, 'metadata', header = False, index = False)
result_df.to_excel(writer, 'data')
writer.save()

# 2.	Предобработка данных

# 3. Анализ данных

# 5. Evaluation

# 6. Deployment